In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import skopt
import scipy
from skopt.space import Real
import category_encoders
from sklearn.svm import SVC
import math

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    
}



In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento.pkl")
test = pd.read_pickle("../Archivos/Arboles_validacion.pkl")
if PREDICCION_REAL:
    test = pd.read_pickle("../Archivos/Arboles_test.pkl").drop(columns = ["Opportunity_ID"])
    test_real = pd.read_pickle("../Archivos/Arboles_test.pkl")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
#fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [4]:
#FECHAS A DIAS

columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date',"Year-Month"]
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [5]:
objetivo = (entrenamiento['Stage'] == 1).astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [6]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 1).astype(int)
    test = test.drop(columns=['Stage'])

In [7]:
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

In [8]:
mean = test["Opportunity_Duration_Ratio"].mean()
test["Opportunity_Duration_Ratio"] = test["Opportunity_Duration_Ratio"].replace(np.NaN,mean)

In [9]:
mean = entrenamiento["Buro_Approved_by_Billing_Country_std"].mean()
entrenamiento["Buro_Approved_by_Billing_Country_std"] = entrenamiento["Buro_Approved_by_Billing_Country_std"].replace(np.NaN,mean)
mean = test["Buro_Approved_by_Billing_Country_std"].mean()
test["Buro_Approved_by_Billing_Country_std"] = test["Buro_Approved_by_Billing_Country_std"].replace(np.NaN,mean)

In [10]:
mean = entrenamiento["Opportunity_Taxable_Rate"].mean()
entrenamiento["Opportunity_Taxable_Rate"] = entrenamiento["Opportunity_Taxable_Rate"].replace(np.NaN,mean)

In [11]:
def limpiar_nan(col):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)

In [12]:
def limpiar_inf(col):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)

In [13]:
limpiar_nan("ASP_by_Billing_Country_std")
limpiar_nan("Total_Amount_by_Billing_Country_std")
limpiar_nan("Opportunity_Duration_by_Product_Family_std")
limpiar_nan("Total_Amount_by_Product_Family_std")
limpiar_nan("Buro_Approved_by_Product_Family")
limpiar_nan("Territory_Duration")
limpiar_nan("Family_Duration")
limpiar_nan("ASP")
limpiar_nan("Opportunity_TRF")
limpiar_nan("Opportunity_Duration_Ratio")
limpiar_nan("Total_Products_Region_Month_Change")
limpiar_nan("Total_Products_Region_Week_Change")
limpiar_inf("Total_Product_Family_Region_Month_Change")
limpiar_nan("Total_Product_Family_Region_Month_Change")

limpiar_inf("Total_Products_Region_Week_Change") #Baja en 0.001 accuracy
limpiar_nan("Total_Products_Region_Week_Change")

limpiar_inf("Total_Products_Region_Month_Change") #Baja en 0.001 accuracy
limpiar_nan("Total_Products_Region_Month_Change")

In [14]:
#entrenamiento = entrenamiento.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])
#test = test.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])

In [15]:
"""
a_usar = ['Sales_Contract_No',
 'Planned_Opportunity_Duration',
 'Account_Name',
 'Product_Name',
 'Bureaucratic_Code',
 'Opportunity_Name',
 'Planned_Time_Until_Deliver',
 'Last_Modified_By',
 'Opportunity_Owner',
 'Product_Family',
 'Account_Type',
 'Territory',
 'Opportunity_Created_Date',
 'Opportunity_Type',
 'Account_Owner',
 'Total_Amount(USD)',
 'Billing_Country',
 'Product_Amount_Deviation_of_Product_Family_rate',
 'Planned_Deliver_Duration',
 'Account_Created_Date']
drop = list(entrenamiento.columns)
for col in a_usar:
    drop.remove(col)
entrenamiento = entrenamiento.drop(columns = drop)    
test = test.drop(columns = drop)   """

"\na_usar = ['Sales_Contract_No',\n 'Planned_Opportunity_Duration',\n 'Account_Name',\n 'Product_Name',\n 'Bureaucratic_Code',\n 'Opportunity_Name',\n 'Planned_Time_Until_Deliver',\n 'Last_Modified_By',\n 'Opportunity_Owner',\n 'Product_Family',\n 'Account_Type',\n 'Territory',\n 'Opportunity_Created_Date',\n 'Opportunity_Type',\n 'Account_Owner',\n 'Total_Amount(USD)',\n 'Billing_Country',\n 'Product_Amount_Deviation_of_Product_Family_rate',\n 'Planned_Deliver_Duration',\n 'Account_Created_Date']\ndrop = list(entrenamiento.columns)\nfor col in a_usar:\n    drop.remove(col)\nentrenamiento = entrenamiento.drop(columns = drop)    \ntest = test.drop(columns = drop)   "

In [16]:
for col in entrenamiento.columns:
    ent_null = entrenamiento.loc[entrenamiento[col] == np.inf]
    if (ent_null[col].count() > 200):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 200):
        print(col)
        print(ent_null[col].count())
        limpiar_inf(col)        
for col in test.columns:
    ent_null = test.loc[test[col] == np.inf]
    if (ent_null[col].count() > 200):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 200):
        print(col)
        print(ent_null[col].count())
        limpiar_inf(col)        

Total_Products_Region_Quarter_Change
4
Region_Total_Product_Family_Region_Month_Change_mean
11982
Region_Total_Products_Region_Week_Change_mean
12081
Region_Total_Products_Region_Month_Change_mean
11982
Region_Total_Products_Region_Quarter_Change_mean
11982
Territory_Total_Product_Family_Region_Month_Change_mean
9646
Territory_Total_Products_Region_Week_Change_mean
10019
Territory_Total_Products_Region_Month_Change_mean
7200
Territory_Total_Products_Region_Quarter_Change_mean
5765
Bureaucratic_Code_Total_Product_Family_Region_Month_Change_mean
11974
Bureaucratic_Code_Total_Products_Region_Week_Change_mean
11771
Bureaucratic_Code_Total_Products_Region_Month_Change_mean
11822
Bureaucratic_Code_Total_Products_Region_Quarter_Change_mean
11771
Source_Total_Product_Family_Region_Month_Change_mean
11969
Source_Total_Products_Region_Week_Change_mean
11611
Source_Total_Products_Region_Month_Change_mean
11291
Source_Total_Products_Region_Quarter_Change_mean
9422
Billing_Country_Total_Product_Fam

In [17]:
for col in entrenamiento.columns:
    ent_null = entrenamiento.loc[entrenamiento[col].isnull()]
    ent_null = ent_null.replace(np.NaN,0)
    if (ent_null[col].count() > 200):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 200):
        print(col)
        print(ent_null[col].count())
        limpiar_nan(col)        
for col in test.columns:
    ent_null = test.loc[test[col].isnull()]
    ent_null = ent_null.replace(np.NaN,0)
    if (ent_null[col].count() > 200):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 200):
        print(col)
        print(ent_null[col].count())
        limpiar_nan(col)        

Quote_Expiry_Date
3015
Total_Products_Region_Quarter_Change
2455
Region_Total_Product_Family_Region_Month_Change_std
12081
Region_Total_Products_Region_Week_Change_std
12081
Region_Total_Products_Region_Month_Change_std
11982
Region_Total_Products_Region_Quarter_Change_std
11982
Territory_ASP_std
11
Territory_ASP_(converted)_std
11
Territory_Total_Amount_std
11
Territory_Total_Taxable_Amount_std
11
Territory_Total_Amount(USD)_std
11
Territory_Total_Taxable_Amount(USD)_std
11
Territory_Total_Product_Family_Region_Last_Month_std
11
Territory_Total_Product_Family_Region_Month_Change_std
9928
Territory_Total_Products_Region_Last_Week_std
11
Territory_Total_Products_Region_Week_Change_std
10039
Territory_Total_Products_Region_Last_Month_std
11
Territory_Total_Products_Region_Month_Change_std
7211
Territory_Total_Products_Region_Last_Quarter_std
11
Territory_Total_Products_Region_Quarter_Change_std
5780
Territory_Opportunity_Total_Amount_std
11
Territory_Planned_Deliver_Duration_std
11
Terri

Account_Name_Planned_Deliver_Duration_std
527
Account_Name_Actual_Opportunity_Duration_std
527
Account_Name_Planned_Time_Until_Deliver_std
527
Account_Name_Planned_Opportunity_Duration_std
527
Account_Name_Opportunity_Taxable_Rate_mean
8
Account_Name_Opportunity_Taxable_Rate_std
540
Account_Name_Product_Amount_Deviation_of_Product_Family_rate_std
527
Account_Name_Opportunity_Duration_Ratio_std
527
Account_Name_Opportunity_Total_Amount_Region_avg_std
527
Account_Name_Opportunity_Total_Amount_Region_std_std
527
Account_Name_Opportunity_Total_Amount_Region_avg_Ratio_std
527
Account_Name_Opportunity_Total_Amount_Region_std_Ratio_std
527
Account_Name_Opportunity_TRF_Region_avg_std
527
Account_Name_Opportunity_TRF_Region_std_std
527
Account_Name_Opportunity_TRF_Region_avg_Ratio_std
527
Account_Name_Opportunity_TRF_Region_std_Ratio_std
527
Account_Name_Opportunity_Duration_by_Billing_Country_std
527
Account_Name_Opportunity_Duration_by_Account_Type_std
527
Account_Name_ASP_by_Region_mean_std


Sales_Contract_No_Opportunity_Duration_by_Product_Family_std_mean
1
Sales_Contract_No_Opportunity_Duration_by_Product_Family_std_std
2416
Sales_Contract_No_Total_Amount_by_Product_Family_mean_std
2414
Sales_Contract_No_Total_Amount_by_Product_Family_std_mean
1
Sales_Contract_No_Total_Amount_by_Product_Family_std_std
2416
Sales_Contract_No_Buro_Approved_by_Product_Family_mean
1
Sales_Contract_No_Buro_Approved_by_Product_Family_std
2416
Sales_Contract_No_Family_Duration_mean
2
Sales_Contract_No_Family_Duration_std
2414
Sales_Contract_No_Territory_Duration_std
2414
Account_Owner_ASP_std
1
Account_Owner_ASP_(converted)_std
1
Account_Owner_Total_Amount_std
1
Account_Owner_Total_Taxable_Amount_std
1
Account_Owner_Total_Amount(USD)_std
1
Account_Owner_Total_Taxable_Amount(USD)_std
1
Account_Owner_Total_Product_Family_Region_Last_Month_std
1
Account_Owner_Total_Product_Family_Region_Month_Change_std
9380
Account_Owner_Total_Products_Region_Last_Week_std
1
Account_Owner_Total_Products_Region_We

Last_Modified_By_ASP_by_Billing_Country_mean_std
3
Last_Modified_By_ASP_by_Billing_Country_std_std
3
Last_Modified_By_Total_Amount_by_Billing_Country_mean_std
3
Last_Modified_By_Total_Amount_by_Billing_Country_std_std
3
Last_Modified_By_Buro_Approved_by_Billing_Country_mean_std
3
Last_Modified_By_Buro_Approved_by_Billing_Country_std_std
3
Last_Modified_By_Opportunity_Duration_by_Product_Family_mean_std
3
Last_Modified_By_Opportunity_Duration_by_Product_Family_std_std
3
Last_Modified_By_Total_Amount_by_Product_Family_mean_std
3
Last_Modified_By_Total_Amount_by_Product_Family_std_std
3
Last_Modified_By_Buro_Approved_by_Product_Family_std
3
Last_Modified_By_Family_Duration_std
3
Last_Modified_By_Territory_Duration_mean
1
Last_Modified_By_Territory_Duration_std
7
Product_Family_ASP_mean
2428
Product_Family_ASP_std
2583
Product_Family_ASP_(converted)_std
31
Product_Family_Total_Amount_std
31
Product_Family_Total_Taxable_Amount_std
31
Product_Family_Total_Amount(USD)_std
31
Product_Family_To

In [18]:
model = SVC(probability = True)

model.fit(entrenamiento,objetivo)
# make the prediction using the resulting model
preds = model.predict(test)
preds

array([0, 1, 1, ..., 1, 1, 1])

In [19]:
if not PREDICCION_REAL:
    print(accuracy_score(test_label, preds))
    print(skl.metrics.log_loss(test_label,preds))

0.9313399778516057
2.371473884790977


In [20]:
model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [21]:
preds = model.predict(entrenamiento)
accuracy_score(objetivo, preds)

0.9538117705487956

In [22]:
if (PREDICCION_REAL):
    preds = model.predict(test)
    resultados = test_real[['Opportunity_ID']].copy()
    resultados['Target'] = pd.Series(preds)
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("prediccion_naive_bayes.csv", index=False)
    resultados['Target'].value_counts()

In [23]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

SystemExit: 

C:\Users\gianb\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from itertools import combinations

In [ ]:
entrenamiento.info()

In [ ]:
columnas = list(entrenamiento.columns)
max_ac = 0
max_col = ""
for col1,col2 in combinations(columnas,2):
    ent_copia = entrenamiento.copy()
    test_copia = test.copy()
    ent_copia.drop(columns = [col1,col2])
    test_copia.drop(columns = [col1,col2])
    model = GaussianNB(var_smoothing = 1e-20)

    model.fit(ent_copia,objetivo)
    # make the prediction using the resulting model
    preds = model.predict(test_copia)
    ac = accuracy_score(test_label, preds)
    if ac >= max_ac:
        max_ac = ac
        max_col = (col1,col2)
        
print(max_ac)
print(max_col)

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold

# Classifier
bayes_cv_tuner = BayesSearchCV(
estimator = skl.naive_bayes.GaussianNB(),
search_spaces = {
        "var_smoothing" : Real(low=1e-30, high=1e-10, prior='uniform')
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 1,
n_iter = 50,
verbose = 1,
refit = True,
random_state = 72
)

In [ ]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [ ]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo, callback=status_print)

In [ ]:
print(bayes_cv_tuner.best_score_)
print(bayes_cv_tuner.best_params_)

In [ ]:
preds = bayes_cv_tuner.predict(test.values)

In [ ]:
skl.metrics.log_loss(test_label,preds)